# Drug Discovery, FAST-AI Tabular Model test


## DATASETS:
(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In [1]:
!nvidia-smi

Fri Apr  8 15:51:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.156.00   Driver Version: 450.156.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   39C    P0    54W / 300W |   8724MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
#%%capture
#!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
#!chmod +x Miniconda3-latest-Linux-x86_64.sh
#!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
#!time conda install -q -y -c conda-forge rdkit

In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem

[15:51:57] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [15:51:57] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
top_mcc_scores = {
    
    'CHEMBL205': 0.862,
    'CHEMBL301': 0.926,
    'CHEMBL240': 0.884,
    'CHEMBL219': 0.887,
    'CHEMBL244': 0.983,
    'CHEMBL218': 0.941,
    'CHEMBL1978': 0.904}

In [6]:
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')

In [7]:
#df.head()
list(path.iterdir())

[PosixPath('../dataset/13321_2017_226_MOESM1_ESM/mol_images'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL219'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL240'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL244'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978')]

In [8]:
dataset='CHEMBL205'

# TabularPandas

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from  sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

In [10]:
from fastai.tabular.all import *

In [11]:
def get_df(dataset, oversample = False):
    df = pd.read_csv(path/f'{dataset}/{dataset}_ecfp_512_activations_32.csv')
    X, y = df.drop(["CID", "SMILES", "Activity", 'Image', 'is_valid'], axis=1), df["Activity"]
    dep_var = ['Activity']
    cat_vars = X.columns
    cat_names = list(cat_vars)
    x_train, x_val = train_test_split(df.index, test_size=0.2, random_state=666, stratify=df['Activity'])
    df.loc[x_train, 'is_valid']=False
    df.loc[x_val, 'is_valid']=True
    if oversample:
        df = pd.concat([*[df.loc[(df.is_valid==False) & (df.Activity==1)]]*10, 
                            *[df.loc[(df.is_valid==False) & (df.Activity==0)]], 
                            df.loc[df.is_valid==True]], 
                            ignore_index=True)
    return df, dep_var, cat_vars, cat_names

In [12]:
df, dep_var, cat_vars, cat_names = get_df(dataset)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17941 entries, 0 to 17940
Columns: 549 entries, CID to pca_31
dtypes: bool(1), float64(32), int64(513), object(3)
memory usage: 75.0+ MB


In [14]:
df.head()

,CID,SMILES,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,ECFP4_8,...,pca_22,pca_23,pca_24,pca_25,pca_26,pca_27,pca_28,pca_29,pca_30,pca_31
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc2)C=2c3c(OC4=CC(=O)C=CC=24)cc(O)cc3)ccc1,1,0,0,0,0,0,0,0,...,-0.053250,0.074476,0.380994,-0.217311,-0.184009,0.222209,-0.092765,0.096560,-0.460699,0.350310
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,0,0,0,0,0,0,0,0,...,0.193729,-0.223540,-0.230245,0.150697,0.096929,-0.012176,-0.161624,-0.146422,-0.220140,-0.208350
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O)C2(C)C,0,0,0,0,0,0,0,0,...,0.086427,-0.170108,-0.065237,-0.146130,0.213724,0.028310,-0.200376,-0.079916,0.170232,-0.009071
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],0,0,0,0,0,0,0,0,...,0.257755,-0.117253,0.223082,0.258841,0.492039,0.259332,0.976807,0.130415,0.025671,-0.478019
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,0,0,0,0,0,0,0,...,0.157034,1.390639,-0.454606,-0.474487,-0.178762,0.688628,0.061859,0.352132,-0.308183,-0.041527


In [15]:
train_idx = np.where(df.is_valid == False)[0]
valid_idx = np.where(df.is_valid == True)[0]
valid_idx

array([    5,    11,    13, ..., 17931, 17934, 17937])

In [16]:
def get_dls(df, dep_var, cat_vars, cat_names):
    train_idx = np.where(df.is_valid == False)[0]
    valid_idx = np.where(df.is_valid == True)[0]
    splits = (list(train_idx), list(valid_idx))
    procs = [Categorify, Normalize]
    to = TabularPandas(df, procs, cat_names=cat_names, y_names=dep_var, splits=splits, y_block = CategoryBlock(),
                      reduce_memory=True)
    to.show(3)
    to.items.head(3)
    dls = to.dataloaders(bs=32)
    return dls

In [17]:
dls = get_dls(df, dep_var, cat_vars, cat_names)

,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,ECFP4_8,ECFP4_9,ECFP4_10,ECFP4_11,ECFP4_12,ECFP4_13,ECFP4_14,ECFP4_15,ECFP4_16,ECFP4_17,ECFP4_18,ECFP4_19,ECFP4_20,ECFP4_21,ECFP4_22,ECFP4_23,ECFP4_24,ECFP4_25,ECFP4_26,ECFP4_27,ECFP4_28,ECFP4_29,ECFP4_30,ECFP4_31,ECFP4_32,ECFP4_33,ECFP4_34,ECFP4_35,ECFP4_36,ECFP4_37,ECFP4_38,ECFP4_39,ECFP4_40,ECFP4_41,ECFP4_42,ECFP4_43,ECFP4_44,ECFP4_45,ECFP4_46,ECFP4_47,ECFP4_48,ECFP4_49,ECFP4_50,ECFP4_51,ECFP4_52,ECFP4_53,ECFP4_54,ECFP4_55,ECFP4_56,ECFP4_57,ECFP4_58,ECFP4_59,ECFP4_60,ECFP4_61,ECFP4_62,ECFP4_63,ECFP4_64,ECFP4_65,ECFP4_66,ECFP4_67,ECFP4_68,ECFP4_69,ECFP4_70,ECFP4_71,ECFP4_72,ECFP4_73,ECFP4_74,ECFP4_75,ECFP4_76,ECFP4_77,ECFP4_78,ECFP4_79,ECFP4_80,ECFP4_81,ECFP4_82,ECFP4_83,ECFP4_84,ECFP4_85,ECFP4_86,ECFP4_87,ECFP4_88,ECFP4_89,ECFP4_90,ECFP4_91,ECFP4_92,ECFP4_93,ECFP4_94,ECFP4_95,ECFP4_96,ECFP4_97,ECFP4_98,ECFP4_99,ECFP4_100,ECFP4_101,ECFP4_102,ECFP4_103,ECFP4_104,ECFP4_105,ECFP4_106,ECFP4_107,ECFP4_108,ECFP4_109,ECFP4_110,ECFP4_111,ECFP4_112,ECFP4_113,ECFP4_114,ECFP4_115,ECFP4_116,ECFP4_117,ECFP4_118,ECFP4_119,ECFP4_120,ECFP4_121,ECFP4_122,ECFP4_123,ECFP4_124,ECFP4_125,ECFP4_126,ECFP4_127,ECFP4_128,ECFP4_129,ECFP4_130,ECFP4_131,ECFP4_132,ECFP4_133,ECFP4_134,ECFP4_135,ECFP4_136,ECFP4_137,ECFP4_138,ECFP4_139,ECFP4_140,ECFP4_141,ECFP4_142,ECFP4_143,ECFP4_144,ECFP4_145,ECFP4_146,ECFP4_147,ECFP4_148,ECFP4_149,ECFP4_150,ECFP4_151,ECFP4_152,ECFP4_153,ECFP4_154,ECFP4_155,ECFP4_156,ECFP4_157,ECFP4_158,ECFP4_159,ECFP4_160,ECFP4_161,ECFP4_162,ECFP4_163,ECFP4_164,ECFP4_165,ECFP4_166,ECFP4_167,ECFP4_168,ECFP4_169,ECFP4_170,ECFP4_171,ECFP4_172,ECFP4_173,ECFP4_174,ECFP4_175,ECFP4_176,ECFP4_177,ECFP4_178,ECFP4_179,ECFP4_180,ECFP4_181,ECFP4_182,ECFP4_183,ECFP4_184,ECFP4_185,ECFP4_186,ECFP4_187,ECFP4_188,ECFP4_189,ECFP4_190,ECFP4_191,ECFP4_192,ECFP4_193,ECFP4_194,ECFP4_195,ECFP4_196,ECFP4_197,ECFP4_198,ECFP4_199,ECFP4_200,ECFP4_201,ECFP4_202,ECFP4_203,ECFP4_204,ECFP4_205,ECFP4_206,ECFP4_207,ECFP4_208,ECFP4_209,ECFP4_210,ECFP4_211,ECFP4_212,ECFP4_213,ECFP4_214,ECFP4_215,ECFP4_216,ECFP4_217,ECFP4_218,ECFP4_219,ECFP4_220,ECFP4_221,ECFP4_222,ECFP4_223,ECFP4_224,ECFP4_225,ECFP4_226,ECFP4_227,ECFP4_228,ECFP4_229,ECFP4_230,ECFP4_231,ECFP4_232,ECFP4_233,ECFP4_234,ECFP4_235,ECFP4_236,ECFP4_237,ECFP4_238,ECFP4_239,ECFP4_240,ECFP4_241,ECFP4_242,ECFP4_243,ECFP4_244,ECFP4_245,ECFP4_246,ECFP4_247,ECFP4_248,ECFP4_249,ECFP4_250,ECFP4_251,ECFP4_252,ECFP4_253,ECFP4_254,ECFP4_255,ECFP4_256,ECFP4_257,ECFP4_258,ECFP4_259,ECFP4_260,ECFP4_261,ECFP4_262,ECFP4_263,ECFP4_264,ECFP4_265,ECFP4_266,ECFP4_267,ECFP4_268,ECFP4_269,ECFP4_270,ECFP4_271,ECFP4_272,ECFP4_273,ECFP4_274,ECFP4_275,ECFP4_276,ECFP4_277,ECFP4_278,ECFP4_279,ECFP4_280,ECFP4_281,ECFP4_282,ECFP4_283,ECFP4_284,ECFP4_285,ECFP4_286,ECFP4_287,ECFP4_288,ECFP4_289,ECFP4_290,ECFP4_291,ECFP4_292,ECFP4_293,ECFP4_294,ECFP4_295,ECFP4_296,ECFP4_297,ECFP4_298,ECFP4_299,ECFP4_300,ECFP4_301,ECFP4_302,ECFP4_303,ECFP4_304,ECFP4_305,ECFP4_306,ECFP4_307,ECFP4_308,ECFP4_309,ECFP4_310,ECFP4_311,ECFP4_312,ECFP4_313,ECFP4_314,ECFP4_315,ECFP4_316,ECFP4_317,ECFP4_318,ECFP4_319,ECFP4_320,ECFP4_321,ECFP4_322,ECFP4_323,ECFP4_324,ECFP4_325,ECFP4_326,ECFP4_327,ECFP4_328,ECFP4_329,ECFP4_330,ECFP4_331,ECFP4_332,ECFP4_333,ECFP4_334,ECFP4_335,ECFP4_336,ECFP4_337,ECFP4_338,ECFP4_339,ECFP4_340,ECFP4_341,ECFP4_342,ECFP4_343,ECFP4_344,ECFP4_345,ECFP4_346,ECFP4_347,ECFP4_348,ECFP4_349,ECFP4_350,ECFP4_351,ECFP4_352,ECFP4_353,ECFP4_354,ECFP4_355,ECFP4_356,ECFP4_357,ECFP4_358,ECFP4_359,ECFP4_360,ECFP4_361,ECFP4_362,ECFP4_363,ECFP4_364,ECFP4_365,ECFP4_366,ECFP4_367,ECFP4_368,ECFP4_369,ECFP4_370,ECFP4_371,ECFP4_372,ECFP4_373,ECFP4_374,ECFP4_375,ECFP4_376,ECFP4_377,ECFP4_378,ECFP4_379,ECFP4_380,ECFP4_381,ECFP4_382,ECFP4_383,ECFP4_384,ECFP4_385,ECFP4_386,ECFP4_387,ECFP4_388,ECFP4_389,ECFP4_390,ECFP4_391,ECFP4_392,ECFP4_393,ECFP4_394,ECFP4_395,ECFP4_396,ECFP4_397,ECFP4_398,ECFP4_399,ECFP4_400,ECFP4_401,ECFP4_402,ECFP4_403,ECFP4_404,ECFP4_405,ECFP4_406,ECFP4_407,ECFP4_408,ECFP4_409,ECFP4_410,ECFP4_4

In [18]:
dls.show_batch()

,ECFP4_1,ECFP4_2,ECFP4_3,ECFP4_4,ECFP4_5,ECFP4_6,ECFP4_7,ECFP4_8,ECFP4_9,ECFP4_10,ECFP4_11,ECFP4_12,ECFP4_13,ECFP4_14,ECFP4_15,ECFP4_16,ECFP4_17,ECFP4_18,ECFP4_19,ECFP4_20,ECFP4_21,ECFP4_22,ECFP4_23,ECFP4_24,ECFP4_25,ECFP4_26,ECFP4_27,ECFP4_28,ECFP4_29,ECFP4_30,ECFP4_31,ECFP4_32,ECFP4_33,ECFP4_34,ECFP4_35,ECFP4_36,ECFP4_37,ECFP4_38,ECFP4_39,ECFP4_40,ECFP4_41,ECFP4_42,ECFP4_43,ECFP4_44,ECFP4_45,ECFP4_46,ECFP4_47,ECFP4_48,ECFP4_49,ECFP4_50,ECFP4_51,ECFP4_52,ECFP4_53,ECFP4_54,ECFP4_55,ECFP4_56,ECFP4_57,ECFP4_58,ECFP4_59,ECFP4_60,ECFP4_61,ECFP4_62,ECFP4_63,ECFP4_64,ECFP4_65,ECFP4_66,ECFP4_67,ECFP4_68,ECFP4_69,ECFP4_70,ECFP4_71,ECFP4_72,ECFP4_73,ECFP4_74,ECFP4_75,ECFP4_76,ECFP4_77,ECFP4_78,ECFP4_79,ECFP4_80,ECFP4_81,ECFP4_82,ECFP4_83,ECFP4_84,ECFP4_85,ECFP4_86,ECFP4_87,ECFP4_88,ECFP4_89,ECFP4_90,ECFP4_91,ECFP4_92,ECFP4_93,ECFP4_94,ECFP4_95,ECFP4_96,ECFP4_97,ECFP4_98,ECFP4_99,ECFP4_100,ECFP4_101,ECFP4_102,ECFP4_103,ECFP4_104,ECFP4_105,ECFP4_106,ECFP4_107,ECFP4_108,ECFP4_109,ECFP4_110,ECFP4_111,ECFP4_112,ECFP4_113,ECFP4_114,ECFP4_115,ECFP4_116,ECFP4_117,ECFP4_118,ECFP4_119,ECFP4_120,ECFP4_121,ECFP4_122,ECFP4_123,ECFP4_124,ECFP4_125,ECFP4_126,ECFP4_127,ECFP4_128,ECFP4_129,ECFP4_130,ECFP4_131,ECFP4_132,ECFP4_133,ECFP4_134,ECFP4_135,ECFP4_136,ECFP4_137,ECFP4_138,ECFP4_139,ECFP4_140,ECFP4_141,ECFP4_142,ECFP4_143,ECFP4_144,ECFP4_145,ECFP4_146,ECFP4_147,ECFP4_148,ECFP4_149,ECFP4_150,ECFP4_151,ECFP4_152,ECFP4_153,ECFP4_154,ECFP4_155,ECFP4_156,ECFP4_157,ECFP4_158,ECFP4_159,ECFP4_160,ECFP4_161,ECFP4_162,ECFP4_163,ECFP4_164,ECFP4_165,ECFP4_166,ECFP4_167,ECFP4_168,ECFP4_169,ECFP4_170,ECFP4_171,ECFP4_172,ECFP4_173,ECFP4_174,ECFP4_175,ECFP4_176,ECFP4_177,ECFP4_178,ECFP4_179,ECFP4_180,ECFP4_181,ECFP4_182,ECFP4_183,ECFP4_184,ECFP4_185,ECFP4_186,ECFP4_187,ECFP4_188,ECFP4_189,ECFP4_190,ECFP4_191,ECFP4_192,ECFP4_193,ECFP4_194,ECFP4_195,ECFP4_196,ECFP4_197,ECFP4_198,ECFP4_199,ECFP4_200,ECFP4_201,ECFP4_202,ECFP4_203,ECFP4_204,ECFP4_205,ECFP4_206,ECFP4_207,ECFP4_208,ECFP4_209,ECFP4_210,ECFP4_211,ECFP4_212,ECFP4_213,ECFP4_214,ECFP4_215,ECFP4_216,ECFP4_217,ECFP4_218,ECFP4_219,ECFP4_220,ECFP4_221,ECFP4_222,ECFP4_223,ECFP4_224,ECFP4_225,ECFP4_226,ECFP4_227,ECFP4_228,ECFP4_229,ECFP4_230,ECFP4_231,ECFP4_232,ECFP4_233,ECFP4_234,ECFP4_235,ECFP4_236,ECFP4_237,ECFP4_238,ECFP4_239,ECFP4_240,ECFP4_241,ECFP4_242,ECFP4_243,ECFP4_244,ECFP4_245,ECFP4_246,ECFP4_247,ECFP4_248,ECFP4_249,ECFP4_250,ECFP4_251,ECFP4_252,ECFP4_253,ECFP4_254,ECFP4_255,ECFP4_256,ECFP4_257,ECFP4_258,ECFP4_259,ECFP4_260,ECFP4_261,ECFP4_262,ECFP4_263,ECFP4_264,ECFP4_265,ECFP4_266,ECFP4_267,ECFP4_268,ECFP4_269,ECFP4_270,ECFP4_271,ECFP4_272,ECFP4_273,ECFP4_274,ECFP4_275,ECFP4_276,ECFP4_277,ECFP4_278,ECFP4_279,ECFP4_280,ECFP4_281,ECFP4_282,ECFP4_283,ECFP4_284,ECFP4_285,ECFP4_286,ECFP4_287,ECFP4_288,ECFP4_289,ECFP4_290,ECFP4_291,ECFP4_292,ECFP4_293,ECFP4_294,ECFP4_295,ECFP4_296,ECFP4_297,ECFP4_298,ECFP4_299,ECFP4_300,ECFP4_301,ECFP4_302,ECFP4_303,ECFP4_304,ECFP4_305,ECFP4_306,ECFP4_307,ECFP4_308,ECFP4_309,ECFP4_310,ECFP4_311,ECFP4_312,ECFP4_313,ECFP4_314,ECFP4_315,ECFP4_316,ECFP4_317,ECFP4_318,ECFP4_319,ECFP4_320,ECFP4_321,ECFP4_322,ECFP4_323,ECFP4_324,ECFP4_325,ECFP4_326,ECFP4_327,ECFP4_328,ECFP4_329,ECFP4_330,ECFP4_331,ECFP4_332,ECFP4_333,ECFP4_334,ECFP4_335,ECFP4_336,ECFP4_337,ECFP4_338,ECFP4_339,ECFP4_340,ECFP4_341,ECFP4_342,ECFP4_343,ECFP4_344,ECFP4_345,ECFP4_346,ECFP4_347,ECFP4_348,ECFP4_349,ECFP4_350,ECFP4_351,ECFP4_352,ECFP4_353,ECFP4_354,ECFP4_355,ECFP4_356,ECFP4_357,ECFP4_358,ECFP4_359,ECFP4_360,ECFP4_361,ECFP4_362,ECFP4_363,ECFP4_364,ECFP4_365,ECFP4_366,ECFP4_367,ECFP4_368,ECFP4_369,ECFP4_370,ECFP4_371,ECFP4_372,ECFP4_373,ECFP4_374,ECFP4_375,ECFP4_376,ECFP4_377,ECFP4_378,ECFP4_379,ECFP4_380,ECFP4_381,ECFP4_382,ECFP4_383,ECFP4_384,ECFP4_385,ECFP4_386,ECFP4_387,ECFP4_388,ECFP4_389,ECFP4_390,ECFP4_391,ECFP4_392,ECFP4_393,ECFP4_394,ECFP4_395,ECFP4_396,ECFP4_397,ECFP4_398,ECFP4_399,ECFP4_400,ECFP4_401,ECFP4_402,ECFP4_403,ECFP4_404,ECFP4_405,ECFP4_406,ECFP4_407,ECFP4_408,ECFP4_409,ECFP4_410,ECFP4_4

# Loss function with weighted classes

In [19]:
def get_loss_func(df):
    TARGET = 'Activity'
    class_count_df = df.groupby(TARGET).count()
    n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]
    w_0 = (n_0 + n_1) / (2.0 * n_0)
    w_1 = (n_0 + n_1) / (2.0 * n_1)
    print(w_0)
    print(w_1)
    class_weights=torch.FloatTensor([w_0, w_1]).cuda()
    loss_func = CrossEntropyLossFlat(weight=class_weights)
    return loss_func

In [20]:
loss_func = get_loss_func(df)

0.55
5.5


# Train tabular DNN model

In [21]:
def train_model(dls, dataset, loss_func=loss_func, epochs=11, wd=0.0002, layers=[200,100]):
    
    print(f'Training model on dataset: {dataset}')
    print(f'Trained epochs: epochs={epochs}')
    
    learn = tabular_learner(dls, 
                            layers=layers,
                            wd=wd,
                            metrics=[accuracy, 
                            F1Score(), 
                            Precision(), 
                            Recall(), 
                            RocAucBinary(), 
                            MatthewsCorrCoef()])
    
    
    print(f'Finding learning rate...')
    lr_min, lr_steep = learn.lr_find(suggestions=True, show_plot=False)
    print(f'Training model with learning rate: {lr_min}')
    learn.fit_one_cycle(epochs, lr_min)
    
    
    return learn

In [ ]:
train_model(dls, dataset, epochs=10)

Training model on dataset: CHEMBL205
Trained epochs: epochs=10
Finding learning rate...


Training model with learning rate: 0.09120108485221863


epoch,train_loss,valid_loss,accuracy,f1_score,precision_score,recall_score,roc_auc_score,matthews_corrcoef,time
0,0.166975,4.006637,0.916133,0.152113,0.931034,0.082822,0.977154,0.263884,02:36
1,0.219104,0.383865,0.912232,0.065282,1.000000,0.033742,0.909828,0.175419,03:01
2,0.268515,0.191934,0.909446,0.006116,1.000000,0.003067,0.911776,0.052817,02:36
3,0.186653,0.359224,0.909167,0.006098,0.500000,0.003067,0.977219,0.033621,02:21


# Test parameters

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {
    'layers': [[200,100], [300,150], [400, 200], [500,250], [600,300], [700, 350], [1000, 500]]
}
param_grid= ParameterGrid(param_grid)

In [ ]:
def train_with_parameters ():
    datasets = ['CHEMBL205_cl', 'CHEMBL301_cl', 'CHEMBL218_cl', 
                'CHEMBL240_cl', 'CHEMBL219_cl', 
                'CHEMBL244_cl', 'CHEMBL1978_cl']
    i = 0
    
    
    df = pd.read_csv(path/f'{datasets[i]}_ecfp_512.csv')
        
    x_train, x_valid = train_test_split(df.index, test_size=0.2, random_state=42, stratify=df['Activity'])
    splits = (list(x_train), list(x_valid))
    procs = [Categorify]
        
    dep_var = ['Activity']
    X = df.drop(["CID", "SMILES", "Activity"], axis=1)
    cat_vars = X.columns
    cat_names = list(cat_vars)
    
    to = TabularPandas(df, 
                        procs,
                        cat_names=cat_names, 
                        y_names=dep_var, 
                        splits=splits, 
                        y_block = CategoryBlock())
    dls = to.dataloaders()
    
    for setting in param_grid:
        
        print(f"Training with layers: {setting['layers']}")
        learn = train_model(dls, datasets[i], layers=setting['layers'])
        

In [ ]:
train_with_parameters()